<a href="https://colab.research.google.com/drive/19LZrxh7qobjTDCWte9WHWzKQUvp0MmaN?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

In [ ]:
!pip install -q kaggle

En el siguiente paso debe subir un token descargado de su perfil de kaggle,  un kaggle.json

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

In [ ]:
! unzip house-prices-advanced-regression-techniques.zip

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('train.csv')
test_data = pd.read_csv("test.csv")

# realizamos preprocesado de los datos, antes de ejecutar los modelos




In [ ]:
feature = np.array(data.columns.to_list())

In [ ]:
nan_val_col = feature[np.array(data.isnull().sum())>146]
nan_val_col

array(['LotFrontage', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence',
       'MiscFeature'], dtype='<U13')

In [ ]:
data_temp = data.drop(nan_val_col, axis=1)
test_data.drop(nan_val_col, axis=1, inplace=True)

In [ ]:
cat_features = np.array([i for i in data_temp.columns.tolist() if data_temp[i].dtype == 'object'])
cont_features = np.array([i for i in data_temp.columns.tolist() if data_temp[i].dtype != 'object'])

In [ ]:
# Eliminando las características que sólo se centraban en una categoría.
data_temp.drop(['SaleCondition', 'SaleType', 'PavedDrive', 'GarageCond', 'GarageQual', 'Functional', 'Electrical',
                'CentralAir', 'Heating', 'BsmtFinType2', 'BsmtCond', 'ExterCond', 'RoofMatl', 'BldgType', 'Condition2',
                'Condition1', 'LandSlope', 'Utilities', 'LandContour', 'Street'], axis=1, inplace=True)

In [ ]:
test_data.drop(['SaleCondition', 'SaleType', 'PavedDrive', 'GarageCond', 'GarageQual', 'Functional', 'Electrical',
                'CentralAir', 'Heating', 'BsmtFinType2', 'BsmtCond', 'ExterCond', 'RoofMatl', 'BldgType', 'Condition2',
                'Condition1', 'LandSlope', 'Utilities', 'LandContour', 'Street'], axis=1, inplace=True)

In [ ]:
#actualizandolas
cat_features = np.array([i for i in data_temp.columns.tolist() if data_temp[i].dtype == 'object'])
cont_features = np.array([i for i in data_temp.columns.tolist() if data_temp[i].dtype != 'object'])

Analizemos los datos continuos


In [ ]:
f,ax = plt.subplots(6,6, figsize=(30,30))
k=0
for i in range(6):
    for j in range(6):
        if k<35:
            ax[i][j].hist(data_temp[cont_features[k]], bins=20, histtype='bar', color='LightGreen')
            ax[i][j].hist(test_data[cont_features[k]], bins=20, histtype='step', color='Red')
            ax[i][j].set_xlabel(cont_features[k])
            ax[i][j].set_ylabel('Count')
            k=k+1

plt.show()

Ahora eliminando categorias que no aportan nada util

In [ ]:
data_temp.drop(['MiscVal', 'PoolArea', 'ScreenPorch', '3SsnPorch',
                'EnclosedPorch', 'KitchenAbvGr', 'BsmtHalfBath','LowQualFinSF',
                'BsmtFinSF2', 'MasVnrArea', 'BsmtFinSF1'], axis=1, inplace=True)

In [ ]:
test_data.drop(['MiscVal', 'PoolArea', 'ScreenPorch', '3SsnPorch',
                'EnclosedPorch', 'KitchenAbvGr', 'BsmtHalfBath','LowQualFinSF',
                'BsmtFinSF2', 'MasVnrArea', 'BsmtFinSF1'], axis=1, inplace=True)

In [ ]:
#Haciendo actualizacion de las categorias
cat_features = np.array([i for i in data_temp.columns.tolist() if data_temp[i].dtype == 'object'])
cont_features = np.array([i for i in data_temp.columns.tolist() if data_temp[i].dtype != 'object'])

resolviendo valores atipicos

In [ ]:
# HAciendo uso del rango intercuartil
def valorAtipico(data_temp, k=3):
    q1, q3 = np.percentile(data_temp, [25, 75])
    
    iqr = q3 - q1
    lower_bound = q1 - (k * iqr)
    upper_bound = q3 + (k * iqr)
    
    valorAtipico = [index for index,x in enumerate(data_temp) if x < lower_bound or x > upper_bound]
    
    return valorAtipico

Numero de valores atipicos estimados

In [ ]:
raws = []
for i in cont_features:
    raws += valorAtipico(data_temp[i])


len(set(raws))

67

In [ ]:
data_temp.drop(list(set(raws)), axis=0, inplace=True)

Datos continuos vs Precio de venta

In [ ]:
f,ax = plt.subplots(6,4, figsize=(28,42))
k=0
for i in range(6):
    for j in range(4):
        if k<22:
            ax[i][j].scatter(x=cont_features[k], y='SalePrice', data=data_temp, s=1, c='red')
            ax[i][j].set_xlabel(cont_features[k])
            ax[i][j].set_ylabel('Sale Price')
            k=k+1
plt.show()

Ahora Separando datos dependientes e independientes

In [ ]:
X = data_temp.iloc[:, :-1]
y = data_temp['SalePrice']

Ahora haciendo la Codificación de los datos categoricos

In [ ]:
#actualizando categorias
cat_features = np.array([i for i in data_temp.columns.tolist() if data_temp[i].dtype == 'object'])
cont_features = np.array([i for i in data_temp.columns.tolist() if data_temp[i].dtype != 'object'])

In [ ]:
# ahora codificando los datos categoricos
from sklearn.preprocessing import LabelEncoder
for i in cat_features:
    # Training Data
    enc = LabelEncoder()
    X[i] =enc.fit_transform(X[i])
    # Test Data
    enc2 = LabelEncoder()
    test_data[i] =enc2.fit_transform(test_data[i])

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(X.corr(), cmap='YlGnBu')

Resolviendo el problema de los Datos nulos

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=10)
X = imputer.fit_transform(X)
test_data = imputer.transform(test_data)

Separando datos de entrenamiento y de test

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X,y, test_size=0.2, random_state=1)

Normalizando variables

In [ ]:
scaler = StandardScaler()

xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

test_data1 = scaler.transform(test_data)

reduccion de informacion irrelevante

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)

xtrain = pca.fit_transform(xtrain)
xtest = pca.transform(xtest)

test_data1 = pca.transform(test_data1)

# Observemos un primer modelo



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

In [ ]:


pipeline1 = Pipeline([
    ('scaler', StandardScaler()),
    ('linearregression', LinearRegression())
])

param_grid = {'linearregression__fit_intercept': [True, False],
              'linearregression__copy_X': [True, False]
              }

grid_search1 = GridSearchCV(estimator=pipeline1, param_grid=param_grid, cv=12)
grid_search1.fit(X, y)

print(f"Best Parameters: {grid_search1.best_params_}\nBest Accuracy Score: {grid_search1.best_score_}")

Best Parameters: {'linearregression__copy_X': True, 'linearregression__fit_intercept': True}
Best Accuracy Score: 0.8859647902124576


In [ ]:
ypred1 = grid_search1.predict(xtest)

sc1 = r2_score(ytest, ypred1)
sc1

Calcula el coeficiente de determinación R2 entre las etiquetas reales (ytest) y las predicciones del modelo (ypred1). El coeficiente R2 indica la proporción de la varianza en la variable dependiente (y) que se puede explicar por el modelo.

# **Arbol de decision**

In [ ]:
from sklearn.tree import DecisionTreeRegressor

pipeline2 = Pipeline([
    ('clf2', DecisionTreeRegressor(random_state=42))
])

param_grid2 = {
    'clf2__max_depth': [2, 3, 4, 5],
    'clf2__min_samples_split': [2, 5, 10],
    'clf2__min_samples_leaf': [1, 2, 4],
    'clf2__max_leaf_nodes': [None, 5, 10, 20]
}

grid_search2 = GridSearchCV(pipeline2, param_grid2, cv=15)
grid_search2.fit(xtrain, ytrain)

print('Parameters : ', grid_search2.best_params_,'\nAccuracy Score : ', grid_search2.best_score_)

In [ ]:
ypred2 = grid_search2.predict(xtest)

sc2 = r2_score(ytest, ypred2)
sc2

# **Random Forest (Bosque aleatorio)**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
pipeline3 = Pipeline([
    ('clf3', RandomForestRegressor())
])

param_grid3 = {
    'clf3__n_estimators': [100, 200, 500],
    'clf3__max_features': ['auto', 'sqrt'],
    'clf3__max_depth': [5, 15],
    'clf3__min_samples_split': [2, 5],
    'clf3__min_samples_leaf': [1, 3],
}

grid_search3 = GridSearchCV(pipeline3, param_grid3, cv=7)
grid_search3.fit(xtrain ,ytrain)

print('Parameters : ', grid_search3.best_params_,'\nAccuracy Score : ', grid_search3.best_score_)

In [ ]:
ypred3 = grid_search3.predict(xtest)

sc3 = r2_score(ytest, ypred3)
sc3

# **Comparacion de modelos**

In [ ]:
plt.plot(['Logisic\nRegression','Decision\nTree','Random\nForest'],
         [grid_search1.best_score_,
          grid_search2.best_score_,
          grid_search3.best_score_,],
        color = 'Orange')

plt.plot(['Logisic\nRegression','Decision\nTree','Random\nForest'],
         [sc1,sc2,sc3],
        color = 'lightgreen')

plt.ylim(0.7,1)
plt.ylabel('Accuracy Score')
plt.xlabel('Classifiers')
plt.title('Model Comparison')
plt.show()

Modelo final

In [ ]:
ypredf = grid_search3.predict(test_data1)